# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from pandas_datareader import wb

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

In [64]:
wb_gini = wb.download(indicator='SI.POV.GINI', start=1990, end=2018)
wb_gini = wb_gini.rename(columns = {'SI.POV.GINI':'GINI'})
wb_gini = wb_gini.reset_index()
wb_gini.year = wb_gini.year.astype(int) # convert year
wb_gini.country = wb_gini.country.astype('string') # convert country to the special pandas string type
wb_gini.info()

wb_gdp = wb.download(indicator='NY.GDP.PCAP.KD', start=1990, end=2018)
wb_gdp = wb_gdp.rename(columns = {'NY.GDP.PCAP.KD':'GDP'})
wb_gdp.reset_index(inplace = True)
wb_gdp.year = wb_gdp.year.astype(int)
wb_gdp.country = wb_gdp.country.astype('string') # convert country to the special pandas string type
wb_gdp.info()
wb_gdp.head()

wb = pd.merge(wb_gini, wb_gdp, how = 'outer', on = ['country','year']);

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  87 non-null     string 
 1   year     87 non-null     int32  
 2   GINI     69 non-null     float64
dtypes: float64(1), int32(1), string(1)
memory usage: 1.8 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  87 non-null     string 
 1   year     87 non-null     int32  
 2   GDP      80 non-null     float64
dtypes: float64(1), int32(1), string(1)
memory usage: 1.8 KB


In [62]:
wb.sample(5)

,country,year,GINI,GDP
41,Mexico,2012,48.7,9401.977909
39,Mexico,2014,48.7,9553.266081
34,Mexico,2019,NaN,10013.703192
93,United States,1992,38.4,39552.021878
52,Mexico,2001,NaN,8782.127396


In [77]:
wb_wgi=pd.read_csv("WB WGI.csv")

# These columns have to go: 'Unnamed: 0' 'Unnamed: 1' 'Unnamed: 2' 'Unnamed: 3'
drop_these = ['Country Code','Series Code'] # use list comprehension to create list of columns
wb_wgi.drop(drop_these, axis=1, inplace=True) # axis = 1 -> columns, inplace=True -> changed, no copy made
wb_wgi.head(5)

col_dict = {}
for i in range(1996, 2022): # range goes from 1996 to 2021
    col_dict[str(i)+" [YR"+str(i)+"]"] = f'wgi{i}' 
col_dict

wb_wgi.rename(columns = col_dict, inplace=True)
wb_wgi.head(7)

,Country Name,Series Name,wgi1996,wgi1998,wgi2000,wgi2002,wgi2003,wgi2004,wgi2005,wgi2006,...,wgi2012,wgi2013,wgi2014,wgi2015,wgi2016,wgi2017,wgi2018,wgi2019,wgi2020,wgi2021
0,Afghanistan,Control of Corruption: Percentile Rank,4.30107545852661,8.02138996124268,4.78723382949829,4.7619047164917,4.7619047164917,6.40394067764282,1.46341466903687,1.95121955871582,...,1.42180097103119,0.947867274284363,5.28846168518066,5.76923084259033,3.8461537361145,3.8461537361145,4.807692527771,6.25,4.807692527771,12.5
1,Afghanistan,Government Effectiveness: Percentile Rank,0,0,0,2.16216206550598,11.3513517379761,18.9054718017578,9.80392169952393,3.90243911743164,...,5.68720388412476,5.68720388412476,7.21153831481934,7.21153831481934,8.17307662963867,8.17307662963867,7.692307472229,7.21153831481934,5.28846168518066,5.76923084259033
2,Afghanistan,Political Stability and Absence of Violence/Te...,2.12765955924988,0.531914889812469,0.52910053730011,1.58730161190033,2.01005029678345,1.94174754619598,2.42718458175659,1.44927537441254,...,1.89573454856873,1.42180097103119,2.85714292526245,0.952380955219269,0.952380955219269,0.476190477609634,0.471698105335236,0.943396210670471,0.471698105335236,1.41509437561035
3,Afghanistan,Regulatory Quality: Percentile Rank,1.63043475151062,2.17391300201416,1.63043475151062,2.16216206550598,4.86486482620239,5.97014904022217,2.94117641448975,2.94117641448975,...,10.4265403747559,10.4265403747559,11.057692527771,14.9038457870483,7.21153831481934,7.21153831481934,10.5769233703613,10.0961542129517,8.17307662963867,8.17307662963867
4,Afghanistan,Rule of Law: Percentile Rank,1.50753772258759,2,1.49253726005554,1.99004971981049,2.98507452011108,2.4038462638855,2.39234447479248,0.478468894958496,...,1.4084507226944,1.877934217453,4.32692289352417,4.32692289352417,5.76923084259033,5.28846168518066,3.8461537361145,4.32692289352417,2.4038462638855,1.92307686805725
5,Afghanistan,Voice and Accountability: Percentile Rank,1,0.497512429952621,0.995024859905243,9.45273590087891,14.4278602600098,15.384614944458,14.9038457870483,17.7884616851807,...,14.0845069885254,14.5539903640747,16.2561569213867,18.7192115783691,20.6896553039551,22.1674880981445,20.2898559570313,21.2560386657715,19.8067626953125,7.24637699127197
6,Albania,Control of Corruption: Percentile Rank,19.3548393249512,18.1818180084229,23.9361705780029,24.3386249542236,22.7513236999512,27.5862064361572,22.9268283843994,22.9268283843994,...,25.1184825897217,24.1706161499023,34.1346168518066,37.5,37.980770111084,41.8269233703613,34.6153831481934,31.730770111084,31.730770111084,31.730770111084


In [79]:
wb_wgi = pd.wide_to_long(wb_wgi, stubnames='wgi', i=['Country Name','Series Name'], j='year')
wb_wgi.sample(10)

ValueError: stubname can't be identical to a column name

In [65]:
# Build up a logical index I
I
for i in range(1990,2022):
    I=wb_wgi.i.str.contains("..")
I
I = wb_wgi.municipality.str.contains('Region')
I |= wb_wgi.municipality.str.contains('Province')
I |= wb_wgi.municipality.str.contains('All Denmark')
empl.loc[I, :]

NameError: name 'I' is not defined

Import your data, either through an API or manually, and load it. 

## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [ ]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.